Importation des bibliothèques

In [1]:
import requests
import pandas as pd

Récupération des données

In [5]:
# Ta clé d'API TMDb
api_key = 'Votre Clé API' #Vous pouvez l'avoir après inscription sur le site de TMDB

# Endpoint pour les films populaires
url = f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page='

# Initialisation des listes pour stocker les données
movie_ids = []
titles = []
popularity = []
overview = []
genres = []
actors = []
directors = []
vote_averages = []
release_years = []

num_pages = 400  # Par exemple, récupérons les 400 premières pages

# Boucle pour récupérer les données de plusieurs pages
for page_num in range(1, num_pages + 1):
    response = requests.get(url + str(page_num))
    if response.status_code == 200:
        data = response.json()
        for movie in data['results']:
            movie_ids.append(movie['id'])
            titles.append(movie['title'])
            popularity.append(movie['popularity'])
            overview.append(movie['overview'])
            
            # Récupération des détails supplémentaires pour chaque film
            movie_detail_url = f"https://api.themoviedb.org/3/movie/{movie['id']}?api_key={api_key}&language=en-US&append_to_response=credits"
            detail_response = requests.get(movie_detail_url)
            if detail_response.status_code == 200:
                movie_detail = detail_response.json()
                # Genres
                genres.append([genre['name'] for genre in movie_detail['genres']])
                # Acteurs (avec vérification)
                if 'credits' in movie_detail and 'cast' in movie_detail['credits']:
                    actors.append([actor['name'] for actor in movie_detail['credits']['cast']])
                else:
                    actors.append(None)
                # Réalisateur
                if 'credits' in movie_detail and 'crew' in movie_detail['credits']:
                    directors.append([crew['name'] for crew in movie_detail['credits']['crew'] if crew['job'] == 'Director'])
                else:
                    directors.append(None)
                # Moyenne des votes
                vote_averages.append(movie_detail['vote_average'])
                # Année de sortie
                release_years.append(pd.to_datetime(movie_detail['release_date']).year)
            else:
                genres.append(None)
                actors.append(None)
                vote_averages.append(None)
                release_years.append(None)
    else:
        print("Erreur lors de la requête")

# Création du DataFrame avec les données récupérées
movies_df = pd.DataFrame({
    'MovieID': movie_ids,
    'Title': titles,
    'Popularity': popularity,
    'Overview': overview,
    'Genres': genres,
    'Actors': actors,
    'Directors': directors,
    'VoteAverage': vote_averages,
    'ReleaseYear': release_years
    
})

# Affichage des premières lignes du DataFrame
movies_df.head()


,MovieID,Title,Popularity,Overview,Genres,Actors,Directors,VoteAverage,ReleaseYear
0,695721,The Hunger Games: The Ballad of Songbirds & Sn...,2222.913,64 years before he becomes the tyrannical pres...,"[Drama, Science Fiction, Action]","[Tom Blyth, Rachel Zegler, Peter Dinklage, Jas...",[Francis Lawrence],7.274,2023.0
1,572802,Aquaman and the Lost Kingdom,1928.895,"Black Manta, still driven by the need to aveng...","[Action, Adventure, Fantasy]","[Jason Momoa, Patrick Wilson, Amber Heard, Yah...",[James Wan],6.500,2023.0
2,1029575,The Family Plan,1463.554,"Dan Morgan is many things: a devoted husband, ...","[Action, Comedy]","[Mark Wahlberg, Michelle Monaghan, Maggie Q, Z...",[Simon Cellan Jones],7.535,2023.0
3,787699,Wonka,950.379,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[Timothée Chalamet, Calah Lane, Keegan-Michael...",[Paul King],7.224,2023.0
4,520758,Chicken Run: Dawn of the Nugget,806.226,A band of fearless chickens flock together to ...,"[Animation, Adventure, Comedy, Family]","[Thandiwe Newton, Zachary Levi, Bella Ramsey, ...",[Sam Fell],7.662,2023.0


In [6]:
# Convertir les listes de genres en chaînes de caractères séparées par des virgules
movies_df['Genres'] = movies_df['Genres'].apply(lambda x: ', '.join(x) if x is not None else '')

# Convertir les listes d'acteurs en chaînes de caractères séparées par des virgules
movies_df['Actors'] = movies_df['Actors'].apply(lambda x: ', '.join(x) if x is not None else '')

# Convertir les listes d'acteurs en chaînes de caractères séparées par des virgules
movies_df['Directors'] = movies_df['Directors'].apply(lambda x: ', '.join(x) if x is not None else '')

# Affichage des premières lignes du DataFrame
movies_df.head()

,MovieID,Title,Popularity,Overview,Genres,Actors,Directors,VoteAverage,ReleaseYear
0,695721,The Hunger Games: The Ballad of Songbirds & Sn...,2222.913,64 years before he becomes the tyrannical pres...,"Drama, Science Fiction, Action","Tom Blyth, Rachel Zegler, Peter Dinklage, Jaso...",Francis Lawrence,7.274,2023.0
1,572802,Aquaman and the Lost Kingdom,1928.895,"Black Manta, still driven by the need to aveng...","Action, Adventure, Fantasy","Jason Momoa, Patrick Wilson, Amber Heard, Yahy...",James Wan,6.500,2023.0
2,1029575,The Family Plan,1463.554,"Dan Morgan is many things: a devoted husband, ...","Action, Comedy","Mark Wahlberg, Michelle Monaghan, Maggie Q, Zo...",Simon Cellan Jones,7.535,2023.0
3,787699,Wonka,950.379,Willy Wonka – chock-full of ideas and determin...,"Comedy, Family, Fantasy","Timothée Chalamet, Calah Lane, Keegan-Michael ...",Paul King,7.224,2023.0
4,520758,Chicken Run: Dawn of the Nugget,806.226,A band of fearless chickens flock together to ...,"Animation, Adventure, Comedy, Family","Thandiwe Newton, Zachary Levi, Bella Ramsey, I...",Sam Fell,7.662,2023.0


In [7]:
# Sauvegarder le DataFrame dans un fichier CSV dans le répertoire de travail actuel
movies_df.to_csv('movies2023.csv', index=False)
# Vérification en affichant quelques lignes du fichier CSV sauvegardé
pd.read_csv('movies2023.csv').head()


,MovieID,Title,Popularity,Overview,Genres,Actors,Directors,VoteAverage,ReleaseYear
0,695721,The Hunger Games: The Ballad of Songbirds & Sn...,2222.913,64 years before he becomes the tyrannical pres...,"Drama, Science Fiction, Action","Tom Blyth, Rachel Zegler, Peter Dinklage, Jaso...",Francis Lawrence,7.274,2023.0
1,572802,Aquaman and the Lost Kingdom,1928.895,"Black Manta, still driven by the need to aveng...","Action, Adventure, Fantasy","Jason Momoa, Patrick Wilson, Amber Heard, Yahy...",James Wan,6.500,2023.0
2,1029575,The Family Plan,1463.554,"Dan Morgan is many things: a devoted husband, ...","Action, Comedy","Mark Wahlberg, Michelle Monaghan, Maggie Q, Zo...",Simon Cellan Jones,7.535,2023.0
3,787699,Wonka,950.379,Willy Wonka – chock-full of ideas and determin...,"Comedy, Family, Fantasy","Timothée Chalamet, Calah Lane, Keegan-Michael ...",Paul King,7.224,2023.0
4,520758,Chicken Run: Dawn of the Nugget,806.226,A band of fearless chickens flock together to ...,"Animation, Adventure, Comedy, Family","Thandiwe Newton, Zachary Levi, Bella Ramsey, I...",Sam Fell,7.662,2023.0


In [10]:
# Affichage des dernières lignes du DataFrame
movies_df.tail()

,MovieID,Title,Popularity,Overview,Genres,Actors,Directors,VoteAverage,ReleaseYear
7995,16392,Redline,14.345,A gorgeous young automobile fanatic--and front...,"Action, Thriller","Nathan Phillips, Nadia Bjorlin, Eddie Griffin,...",Andy Cheng,5.000,2007.0
7996,5203,Wild Orchid,16.343,A woman lawyer becomes mesmerized by a self-ma...,"Romance, Drama","Jacqueline Bisset, Carré Otis, Mickey Rourke, ...",Zalman King,5.236,1989.0
7997,93837,So Undercover,15.296,When the FBI hires her to go undercover at a c...,"Action, Comedy","Miley Cyrus, Jeremy Piven, Mike O'Malley, Josh...",Tom Vaughan,6.160,2012.0
7998,26581,Dr. Who and the Daleks,11.914,Scientist Doctor Who accidentally activates hi...,"Adventure, Family, Science Fiction","Peter Cushing, Roy Castle, Jennie Linden, Robe...",Gordon Flemyng,5.700,1965.0
7999,29143,Eyes of Laura Mars,17.093,A famous fashion photographer develops a distu...,"Mystery, Thriller","Faye Dunaway, Tommy Lee Jones, Brad Dourif, Re...",Irvin Kershner,5.800,1978.0
